# Covid-19 Deaths with Underlying Conditions

In [1]:
# importing packages
import pandas as pd

#set max rows on pd df
pd.set_option('display.max_rows', None)

## Import Csv File as a Dataframe

In [2]:
#import csv
df = pd.read_csv(r"C:\Users\Justin Schwerdt\Documents\Data_Analytics_Projects\Covid_Deaths\Covid_Death_Contributing_Conditions.csv")

## Inspect Original Dataframe

In [3]:
#df initial inspection
df.head()

,Sample Start Date,Sample End Date,Grouped By,Year,Month,State,Condition Group,Condition,Age Group,COVID-19 Deaths
0,1/1/2020,3/25/2023,By Total,NaN,NaN,United States,Respiratory diseases,Influenza and pneumonia,0-24,1526.0
1,1/1/2020,3/25/2023,By Total,NaN,NaN,United States,Respiratory diseases,Influenza and pneumonia,25-34,5745.0
2,1/1/2020,3/25/2023,By Total,NaN,NaN,United States,Respiratory diseases,Influenza and pneumonia,35-44,14969.0
3,1/1/2020,3/25/2023,By Total,NaN,NaN,United States,Respiratory diseases,Influenza and pneumonia,45-54,37187.0
4,1/1/2020,3/25/2023,By Total,NaN,NaN,United States,Respiratory diseases,Influenza and pneumonia,55-64,81992.0


## Remove Useless Rows

In [4]:
''' In this dataframe, we have a grouped by column. This creates redundant information. We only need columns that are grouped
by 'month' to simplify the data. We can use aggregate functions to group by year and total '''

# removing grouped by year and total
new_df = df[df['Grouped By'] == 'By Month']

## Removing Useless Columns

In [5]:
'''Now we can delete the 'Grouped by' column''' 

# removing grouped by column
new_df = new_df.drop(columns='Grouped By')

new_df.head()

,Sample Start Date,Sample End Date,Year,Month,State,Condition Group,Condition,Age Group,COVID-19 Deaths
62100,1/1/2020,1/31/2020,2020.0,1.0,United States,Respiratory diseases,Influenza and pneumonia,0-24,0.0
62101,2/1/2020,2/29/2020,2020.0,2.0,United States,Respiratory diseases,Influenza and pneumonia,0-24,0.0
62102,3/1/2020,3/31/2020,2020.0,3.0,United States,Respiratory diseases,Influenza and pneumonia,0-24,9.0
62103,4/1/2020,4/30/2020,2020.0,4.0,United States,Respiratory diseases,Influenza and pneumonia,0-24,27.0
62104,5/1/2020,5/31/2020,2020.0,5.0,United States,Respiratory diseases,Influenza and pneumonia,0-24,19.0


## Dealing with Null Values

In [6]:
#checking sum of null values
new_df.isnull().sum()

Sample Start Date         0
Sample End Date           0
Year                      0
Month                     0
State                     0
Condition Group           0
Condition                 0
Age Group                 0
COVID-19 Deaths      148733
dtype: int64

In [7]:
''' When there is no value for recorded covid deaths, its safe to assume the value was 0 '''

# filling na values
new_df['COVID-19 Deaths'] = new_df['COVID-19 Deaths'].fillna(0)

## Standardizing Data Types

In [8]:
new_df.dtypes

Sample Start Date     object
Sample End Date       object
Year                 float64
Month                float64
State                 object
Condition Group       object
Condition             object
Age Group             object
COVID-19 Deaths      float64
dtype: object

In [9]:
''' As we can see, Month and Year columns are floats, we want an ints. Also Covid Deaths can be an int. 
You can't have .5 deaths. '''

#adjusting data types
new_df['Year'] = new_df['Year'].astype('int64')
new_df['Month'] = new_df['Month'].astype('int64')
new_df['COVID-19 Deaths'] = new_df['COVID-19 Deaths'].astype('int64')

## Export Our New Data Set as a CSV

In [10]:
# export to csv
new_df.to_csv(r"C:\Users\Justin Schwerdt\Documents\Data_Analytics_Projects\Covid_Deaths\Covid_Death_Contributing_Conditions_Cleaned.csv", index=False)

# Analysis

## Underlying Condition Risk

In [11]:
# get a list of unique values from 'Condition Group'
new_df['Condition Group'].unique()

array(['Respiratory diseases', 'Circulatory diseases', 'Sepsis',
       'Malignant neoplasms', 'Diabetes', 'Obesity', 'Alzheimer disease',
       'Vascular and unspecified dementia', 'Renal failure',
       'Intentional and unintentional injury, poisoning, and other adverse events',
       'All other conditions and causes (residual)', 'COVID-19'],
      dtype=object)

In [12]:
# total of all covid deaths
new_df['COVID-19 Deaths'].sum()

16807214

In [13]:
# checking covid deaths per underlying condition type
new_df.groupby(['Condition Group']).agg({'COVID-19 Deaths':['sum']})

,COVID-19 Deaths
,sum
Condition Group,
All other conditions and causes (residual),1838890
Alzheimer disease,123381
COVID-19,4476985
Circulatory diseases,3039957
Diabetes,640378
"Intentional and unintentional injury, poisoning, and other adverse events",98072
Malignant neoplasms,251426
Obesity,189605


In [27]:
# Underlying Conditions by %
new_df.groupby(['Condition Group']).agg({'COVID-19 Deaths':['sum']}) / new_df['COVID-19 Deaths'].sum() * 100

,COVID-19 Deaths
,sum
Condition Group,
All other conditions and causes (residual),10.941076
Alzheimer disease,0.734095
COVID-19,26.637282
Circulatory diseases,18.087215
Diabetes,3.810138
"Intentional and unintentional injury, poisoning, and other adverse events",0.583511
Malignant neoplasms,1.495941
Obesity,1.128117


In [32]:
''' From this data, we can see that Respiratory disease patients are at a large risk. Knowing that, we will want to look into
which forms of respiratory diseases are at the highest risk '''

# look at sum of deaths per Respiratory disease conditions
new_df.loc[new_df['Condition Group'] == 'Respiratory diseases'].groupby(['Condition']).agg({'COVID-19 Deaths':['sum']})

,COVID-19 Deaths
,sum
Condition,
Adult respiratory distress syndrome,401753
Chronic lower respiratory diseases,402030
Influenza and pneumonia,2079390
Other diseases of the respiratory system,217095
Respiratory arrest,76542
Respiratory failure,1696489


In [33]:
# look at deaths per Respiratory disease conditions %
new_df.loc[new_df['Condition Group'] == 'Respiratory diseases'].groupby(['Condition']).agg({'COVID-19 Deaths':['sum']}) / new_df['COVID-19 Deaths'].sum() * 100

,COVID-19 Deaths
,sum
Condition,
Adult respiratory distress syndrome,2.390360
Chronic lower respiratory diseases,2.392009
Influenza and pneumonia,12.372009
Other diseases of the respiratory system,1.291677
Respiratory arrest,0.455412
Respiratory failure,10.093814


## Age Group Risk

In [34]:
# total deaths per age group
new_df.groupby(['Age Group']).agg({'COVID-19 Deaths':['sum']})

,COVID-19 Deaths
,sum
Age Group,
0-24,16436
25-34,63253
35-44,180858
45-54,482752
55-64,1162534
65-74,1926027
75-84,2217425
85+,2162934


In [36]:
#percentage of deaths per age group
new_df.groupby(['Age Group']).agg({'COVID-19 Deaths':['sum']}) / new_df['COVID-19 Deaths'].sum() * 100

,COVID-19 Deaths
,sum
Age Group,
0-24,0.097791
25-34,0.376344
35-44,1.076074
45-54,2.872290
55-64,6.916875
65-74,11.459526
75-84,13.193293
85+,12.869081
